### Libraries:

In [1]:
#Import the necessary libraries
import os
import timeit

import pandas as pd
import numpy as np
from numpy import mean #, std
# from numpy.random import randn, seed

import folium
# from folium import plugins
from folium.plugins import HeatMap
from folium import IFrame
# import seaborn as sns
# import gmplot

import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.ticker as ticker
from matplotlib.ticker import PercentFormatter

# import tkinter

#For improved table display in the notebook
# from IPython.display import display
pd.options.display.max_columns = None
pd.options.display.max_rows = None

# DON'T READ THIS PART

### Reading the dataset:

In [2]:
Data=pd.read_csv("C:/Users/Victor.SalasAranda/Desktop/FSA/DATA/_business_data_fhrs-inspection__view_detail.csv")

C:\ProgramData\Anaconda2019\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (3,16,18,27,29,30,31,32,33,34,36) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
Data.tail()

,@id,type,establishment,establishment.establishmentRegistration.authorityEstablishmentID,establishment.establishmentRegistration.fhrsEstablishmentStatus.prefLabel,establishment.establishmentRegistration.enrolmentStatus.prefLabel,establishment.tradingName,establishment.establishmentType.label,establishment.establishmentRN,establishment.visibility.notation,establishment.premises.prefAddress.lat,establishment.premises.prefAddress.long,establishment.premises.prefAddress.uprn,establishment.premises.prefAddress.extendedAddress,inspectionAuthority.notation,inspectionAuthority.prefLabel,inspectionAuthority.legacyAuthorityCode,context.tradingName,context.operatorName,inspectionDateTime,rating,rating.notation,summaryScore,hygieneScore,structuralScore,confidenceScore,publicationDateTime,isAppeal,appealStatus.notation,lodgingDateTime,determinationDateTime,exercisedRightToReply,isOnHold,isRescore,originalInspectionDateTime,originalSummaryScore,inspectionPriorToSchemeStartDate,inspectionProcessStatus.label,inspectionProcessStatus.notation,inspectionScheme,inspectionScheme.prefLabel,hasStart,hasEnd
2671996,http://data.food.gov.uk/business/id/establishm...,http://data.food.gov.uk/business/def/core/Insp...,http://data.food.gov.uk/business/id/establishm...,19/00138/FMAN,excluded,active,NaN,Manufacturers and packers,NaN,notpublic,NaN,NaN,NaN,Address redacted,4261,Chiltern District Council,22,Griffiths Brothers Ltd,Griffiths Brothers Distillery Ltd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Awaiting inspection,awaiting-inspection,http://data.food.gov.uk/business/def/fhrs/fhrs...,FHRS,2019-11-12T16:04:45,NaN
2671997,http://data.food.gov.uk/business/id/establishm...,http://data.food.gov.uk/business/def/core/Insp...,http://data.food.gov.uk/business/id/establishm...,19/00158/FPUBFC,awaiting-inspection,active,NaN,Pub or Club,NaN,public,NaN,NaN,NaN,"Chalfont Station Road, Little Chalfont, Buckin...",4261,Chiltern District Council,22,The Metropolitan,The Metropolitan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-11-12T00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Awaiting inspection,awaiting-inspection,http://data.food.gov.uk/business/def/fhrs/fhrs...,FHRS,2019-11-12T16:04:45,NaN
2671998,http://data.food.gov.uk/business/id/establishm...,http://data.food.gov.uk/business/def/core/Insp...,http://data.food.gov.uk/business/id/establishm...,10090/2008/2/000,included,active,NaN,Caring Establishments,NaN,public,5.1676605224609375E1,-5.855739116668701E-1,100081079107,"PRATT HOUSE, QUILL HALL LANE, AMERSHAM, BUCKIN...",4261,Chiltern District Council,22,The Abbeyfield Society,Abbeyfield,2019-10-01T00:00:00,http://data.food.gov.uk/business/def/fhrs/fhrs-1,fhrs-1,1.0,15.0,10.0,20.0,2019-11-12T00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://data.food.gov.uk/business/def/fhrs/fhrs...,FHRS,2019-11-12T16:04:45,NaN
2671999,http://data.food.gov.uk/business/id/establishm...,http://data.food.gov.uk/business/def/core/Insp...,http://data.food.gov.uk/business/id/establishm...,16/00111/FCAFE,included,active,NaN,Restaurant or Café or Canteen,NaN,public,5.160257339477539E1,-5.654237866401672E-1,100081083281,"GOLD HILL WEST, CHALFONT ST. PETER, GERRARDS C...",4261,Chiltern District Council,22,Jolly Farmer,The White Brasserie Co Ltd,2019-11-11T00:00:00,http://data.food.gov.uk/business/def/fhrs/fhrs-5,fhrs-5,5.0,5.0,0.0,0.0,2019-11-12T00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://data.food.gov.uk/business/def/fhrs/fhrs...,FHRS,2019-11-12T16:04:45,NaN
2672000,http://data.food.gov.uk/business/id/establishm...,http://data.food.gov.uk/business/def/core/Insp...,http://data.food.gov.uk/business/id/establishm...,19/00167/FHCAT,awaiting-inspection-and-private,active,NaN,Restaurants and caterers - Other,NaN,private,NaN,NaN,NaN,Address redacted,4261,Chiltern District Council,22,The Versatle Chef Ltd,Fabio Vieira,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-11-12T00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Awaiting inspection,awaiting-inspection,ht

In [4]:
Data.columns

Index(['@id', 'type', 'establishment',
       'establishment.establishmentRegistration.authorityEstablishmentID',
       'establishment.establishmentRegistration.fhrsEstablishmentStatus.prefLabel',
       'establishment.establishmentRegistration.enrolmentStatus.prefLabel',
       'establishment.tradingName', 'establishment.establishmentType.label',
       'establishment.establishmentRN', 'establishment.visibility.notation',
       'establishment.premises.prefAddress.lat',
       'establishment.premises.prefAddress.long',
       'establishment.premises.prefAddress.uprn',
       'establishment.premises.prefAddress.extendedAddress',
       'inspectionAuthority.notation', 'inspectionAuthority.prefLabel',
       'inspectionAuthority.legacyAuthorityCode', 'context.tradingName',
       'context.operatorName', 'inspectionDateTime', 'rating',
       'rating.notation', 'summaryScore', 'hygieneScore', 'structuralScore',
       'confidenceScore', 'publicationDateTime', 'isAppeal',
       'appealSt

### Filtering by features we need and cleaning the empty values:

Features:

- 'context.tradingName' = Business Name

- 'summaryScore' = Rating Value

- 'establishment.establishmentType.label' = Business Type Description

- 'establishment.premises.prefAddress.long' = Longitude

- 'establishment.premises.prefAddress.lat' = Latitude

- 'inspectionAuthority.notation' = Local Authority id

- 'inspectionAuthority.prefLabel' = Local Authority name

In [ ]:
data = Data.loc[:, ['context.tradingName', 'summaryScore', 'establishment.establishmentType.label',
                    'establishment.premises.prefAddress.long', 'establishment.premises.prefAddress.lat',
                   'inspectionAuthority.notation','inspectionAuthority.prefLabel']]

data.columns = ['Business Name', 'Rating Value', 'Business Type Description',
                    'Longitude', 'Latitude', 'Local Authority id', 'Local Authority name']
data = data.dropna()
a=[list(range(0,len(data)))]
data=data.set_index(a)
data.head()

### Fixing long and lat problem:

We have to elimate next issues:

- Delete duplicates and triplicates (e.g: data['Latitude'][7639] (duplicated) and data['Latitude'][168017] (triplicated)):


In [ ]:
l=[]
l2=[]
for i in range(len(data)):
    n=len(data['Longitude'][i])
    m=len(data['Latitude'][i])
    if '|' in data['Longitude'][i]: #for duplicated and triplicated values
        k1=data['Longitude'][i].index('|')
        k2=data['Latitude'][i].index('|')
        l.append(data['Longitude'][i][:k1]) #for normal values
        l2.append(data['Latitude'][i][:k2])      
    else:
        l.append(data['Longitude'][i][:n]) #for normal values
        l2.append(data['Latitude'][i][:m]) 
data['Latitude2']=l2
data['Longitude2']=l

- E0 and E1 for long&la, change scientific notation to normal one.

In [ ]:
data['Latitude2'] = data['Latitude2'].astype(float)
data['Longitude2'] = data['Longitude2'].astype(float)
data['Rating Value']=data['Rating Value'].astype('int')

Rename and filter the columns:

In [ ]:
data = data.loc[:,['Business Name', 'Rating Value', 'Business Type Description',
                    'Latitude2', 'Longitude2','Local Authority id', 'Local Authority name']]
data.columns = ['Business Name', 'Rating Value', 'Business Type Description',
                    'Latitude','Longitude','Local Authority id', 'Local Authority name']
data.head()
# data.to_csv('map2.csv')

# START READING HERE

### Visualizations

#### Loading dataset

In [96]:
tic = timeit.default_timer();
data=pd.read_csv("map2.csv")
#data=pd.read_csv("Q:/Inform/Strategic Surveillance/Sprint AI/DATA/All Features Datasets/map2.csv")
toc = timeit.default_timer()
print ('Elapsed: %s' % (toc-tic))

Elapsed: 36.952877499999886


In [97]:
data['Rating Value']=data['Rating Value'].astype('int')
data['Latitude']=data['Latitude'].astype('float')
data['Longitude']=data['Longitude'].astype('float')

In [98]:
data.head()

,Unnamed: 0,Business Name,Rating Value,Business Type Description,Latitude,Longitude,Local Authority id,Local Authority name
0,0,St John Lloyd Primary,5,Caring Establishments,51.514271,-3.108875,8015,City of Cardiff Council|Cyngor Dinas Caerdydd
1,1,St John Lloyd Primary,5,Caring Establishments,51.514271,-3.108875,8015,City of Cardiff Council|Cyngor Dinas Caerdydd
2,2,The Lotus House,1,Take-away,51.508640,-3.124142,8015,City of Cardiff Council|Cyngor Dinas Caerdydd
3,3,First Cafes Ltd,4,Restaurants and caterers - Other,51.498825,-3.119344,8015,City of Cardiff Council|Cyngor Dinas Caerdydd
4,4,First Cafes Ltd,5,Restaurants and caterers - Other,51.498825,-3.119344,8015,City of Cardiff Council|Cyngor Dinas Caerdydd


In [99]:
data['Business Type Description'].unique().tolist()

['Caring Establishments',
 'Take-away',
 'Restaurants and caterers - Other',
 'Pub or Club',
 'Restaurant or Café or Canteen',
 'School or College',
 'Supermarket or Hypermarket',
 'Manufacturers and packers',
 'Mobile food unit',
 'Retailers - Others',
 'Smaller retailers',
 'Distributors or Transporters',
 'Hotel or Guest House',
 'Importers or Exporters',
 'Primary producer']

#### Business Type filter:

Filtering:

In [317]:
BussTyp=data.loc[data['Business Type Description']== 'Caring Establishments']
# BussTyp=data.loc[data['Business Type Description']== 'Take-away']
# BussTyp=data.loc[data['Business Type Description']== 'Restaurants and caterers - Other']
# BussTyp=data.loc[data['Business Type Description']== 'Pub or Club']
# BussTyp=data.loc[data['Business Type Description']== 'Restaurant or Café or Canteen']
# BussTyp=data.loc[data['Business Type Description']== 'School or College']
# BussTyp=data.loc[data['Business Type Description']== 'Supermarket or Hypermarket']
# BussTyp=data.loc[data['Business Type Description']== 'Manufacturers and packers']
# BussTyp=data.loc[data['Business Type Description']== 'Mobile food unit']
# BussTyp=data.loc[data['Business Type Description']== 'Retailers - Others']
# BussTyp=data.loc[data['Business Type Description']== 'Smaller retailers']
# BussTyp=data.loc[data['Business Type Description']== 'Distributors or Transporters']
# BussTyp=data.loc[data['Business Type Description']== 'Hotel or Guest House']
# BussTyp=data.loc[data['Business Type Description']== 'Importers or Exporters']
# BussTyp=data.loc[data['Business Type Description']== 'Primary producer']

a=[list(range(0,len(BussTyp)))]
BussTyp=BussTyp.set_index(a)

In [318]:
BussTyp.head()

,Unnamed: 0,Business Name,Rating Value,Business Type Description,Latitude,Longitude,Local Authority id,Local Authority name
0,0,St John Lloyd Primary,5,Caring Establishments,51.514271,-3.108875,8015,City of Cardiff Council|Cyngor Dinas Caerdydd
1,1,St John Lloyd Primary,5,Caring Establishments,51.514271,-3.108875,8015,City of Cardiff Council|Cyngor Dinas Caerdydd
2,53,Chapel Break Pre-School & Out Of School Clubs,5,Caring Establishments,52.640305,1.212659,4091,Norwich City Council
3,58,Cardiff & Vale Nursery (Trowbridge Site),4,Caring Establishments,51.513325,-3.109990,8015,City of Cardiff Council|Cyngor Dinas Caerdydd
4,59,Meithrinfa Glan Hafren Nursery,4,Caring Establishments,51.513325,-3.109990,8015,City of Cardiff Council|Cyngor Dinas Caerdydd


#### Global rating average:

In [319]:
Bj=data['Rating Value'].mean()
Bj

4.067063416869742

#### Business Type   rating average:

If you want to compare with the LA's B, delete the #'s:

In [320]:
#Bj=BussTyp['Rating Value'].mean()
#Bj

Number of Local Authority id in TakeAway dataset:

In [321]:
len(BussTyp['Local Authority id'].unique().tolist())

349

#### Creating the LA dataset

Creating a key dataframe to merge the Local Authority name:

In [322]:
dfD = BussTyp[['Local Authority id','Local Authority name']]
dfD = dfD.drop_duplicates()

Merging:

In [323]:
a=10000 #size of the 10 biggest  and 10 smallest values
b=a 
c=1500 #size of the normal values
df = BussTyp.groupby('Local Authority id', as_index=False)['Rating Value'].mean()
df=pd.merge(df,dfD[['Local Authority id','Local Authority name']], on= 'Local Authority id', how='right')
df['Latitude'] = BussTyp.groupby('Local Authority id', as_index=False)['Latitude'].mean()['Latitude'].tolist()
df['Longitude'] = BussTyp.groupby('Local Authority id', as_index=False)['Longitude'].mean()['Longitude'].tolist()
df['C'] = df['Rating Value'] - Bj 
df2=df.nlargest(10,'C')
df2=df2.loc[df2['C']>0]
df2['Size']=a
df3=pd.merge(df, df2[['Local Authority id','Size']], on= 'Local Authority id', how='left')

for i in range(len(df3)):
    if df3['Size'][i]!=a:
        df3['Size'][i]=c
df3['Size']=df3['Size'].astype('int')
df=df3
df.head(n=72)
df2=df.nsmallest(10,'C')
df2=df2.loc[df2['C']<0]
df2['Size2']=b
df3=pd.merge(df, df2[['Local Authority id','Size2']], on= 'Local Authority id', how='left')
df=df3
df.head(n=72)
for i in range(len(df3)):
    if df3['Size2'][i]==b:
        df3['Size'][i]=b
df = df3.drop(columns = ['Size2'])
df.head(n=72)


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Local Authority id,Rating Value,Local Authority name,Latitude,Longitude,C,Size
0,4000,4.556332,Birmingham City Council,52.478805,-1.887220,0.489269,1500
1,4001,4.592187,Borough Council of Kings Lynn and West Norfolk,52.755831,0.429204,0.525124,1500
2,4003,4.800000,City of London Corporation,51.519234,-0.097120,0.732937,1500
3,4004,4.510703,Worthing Borough Council,50.818614,-0.385976,0.443640,1500
4,4005,4.674814,Mid Sussex District Council,51.026982,-0.094652,0.607751,1500
5,4006,4.718750,Horsham District Council,51.034012,-0.344650,0.651687,1500
6,4007,4.731449,Crawley Borough Council,51.119243,-0.180979,0.664385,1500
7,4008,4.567989,Chichester District Council,50.848933,-0.792414,0.500925,1500
8,4009,4.608558,Arun District Council,50.803684,-0.599870,0.541494,1500
9,4010,4.554945,Adur District Council,50.832867,-0.282156,0.487882,1500


In [326]:
df['C'].describe()

count    349.000000
mean       0.541414
std        0.195592
min       -0.913217
25%        0.443640
50%        0.574499
75%        0.674316
max        0.856014
Name: C, dtype: float64

#### Bubble map

In [327]:
import branca.colormap as cm
rangeFrac = 0.8
A=abs(df['C'].min())
B=abs(df['C'].max())
l=[A,B]
C=0.15*max(l)
linear = cm.LinearColormap(['tomato','white','royalblue'], vmin=-C, vmax=C)
linear2 = cm.LinearColormap(['tomato','royalblue'], vmin=-C, vmax=C)
linear2 = linear2.to_step(index=[-C, 0, C])
linear2

In [328]:
# import the library
import folium
import pandas as pd
#https://python-visualization.github.io/folium/modules.html

m = folium.Map(location=[51.9142707824707,-1.108874797821045],zoom_start = 7) 
# I can add marker one by one on the map
for i in range(0,len(df)):
    if df.iloc[i]['Size']!=a:
        color= linear(df.iloc[i]['C'])
    else:
        color= linear2(df.iloc[i]['C']),
    folium.Circle(
      location=[df.iloc[i]['Latitude'], df.iloc[i]['Longitude']],
      #popup=data.iloc[i]['name'],
      popup=folium.Popup("<b>"+df.iloc[i]['Local Authority id'].astype('int').astype('str')+': '+ df.iloc[i]['Local Authority name'] + 
                         "</b> <br> C: "+ round(df.iloc[i]['C'],4).astype('str'),
                         max_width=2650),
      radius= df.iloc[i]['Size'].astype('float'),
      fillColor= linear(df.iloc[i]['C']),
      #fillOpacity=,
      #opacity 
      color=color,
      fill=True,
   ).add_to(m)

title_html = '''
             <h3 align="center" style="font-size:20px"><b>Your map title</b></h3>
             '''
m.get_root().html.add_child(folium.Element(title_html))

m

In [253]:
df.nsmallest(10,'C')

,Local Authority id,Rating Value,Local Authority name,Latitude,Longitude,C,Size
284,4326,3.153846,Council of the Isles of Scilly,49.913703,-6.309266,-0.913217,10000
71,4088,3.923077,Corby Borough Council,52.490464,-0.695898,-0.143986,10000
232,4273,4.025074,Wigan Metropolitan Borough Council,53.525872,-2.594408,-0.041990,10000
346,8019,4.059701,Blaenau Gwent County Borough Council|Cyngor Bw...,51.767384,-3.194358,-0.007362,10000
158,4185,4.063622,London Borough of Harrow,51.593192,-0.338802,-0.003442,10000
329,8002,4.067633,Vale of Glamorgan Council|Cyngor Bro Morgannwg,51.426300,-3.288572,0.000569,10000
165,4192,4.069930,London Borough of Enfield,51.640714,-0.087723,0.002867,10000
100,4121,4.090625,Hyndburn Borough Council,53.762028,-2.385604,0.023562,10000
163,4190,4.120185,London Borough of Havering,51.571017,0.207613,0.053121,10000
332,8005,4.132184,Powys County Council|Cyngor Sir Powys,52.251450,-3.335785,0.065120,10000


### To save as html each one:

#### Comparing with Global rating average

['Caring Establishments',
 'Take-away',
 'Restaurants and caterers - Other',
 'Pub or Club',
 'Restaurant or Café or Canteen',
 'School or College',
 'Supermarket or Hypermarket',
 'Manufacturers and packers',
 'Mobile food unit',
 'Retailers - Others',
 'Smaller retailers',
 'Distributors or Transporters',
 'Hotel or Guest House',
 'Importers or Exporters',
 'Primary producer']

In [329]:
for j in data['Business Type Description'].unique().tolist():
    BussTyp=data.loc[data['Business Type Description']== j]
    a2=[list(range(0,len(BussTyp)))]
    BussTyp=BussTyp.set_index(a2)
    Bj=data['Rating Value'].mean()
    dfD = BussTyp[['Local Authority id','Local Authority name']]
    dfD = dfD.drop_duplicates()
    
    a=10000 #size of the 10 biggest  and 10 smallest values
    b=a 
    c=1500 #size of the normal values
    df = BussTyp.groupby('Local Authority id', as_index=False)['Rating Value'].mean()
    df=pd.merge(df,dfD[['Local Authority id','Local Authority name']], on= 'Local Authority id', how='right')
    df['Latitude'] = BussTyp.groupby('Local Authority id', as_index=False)['Latitude'].mean()['Latitude'].tolist()
    df['Longitude'] = BussTyp.groupby('Local Authority id', as_index=False)['Longitude'].mean()['Longitude'].tolist()
    df['C'] = df['Rating Value'] - Bj 
    df2=df.nlargest(10,'C')
    df2=df2.loc[df2['C']>0]
    df2['Size']=a
    df3=pd.merge(df, df2[['Local Authority id','Size']], on= 'Local Authority id', how='left')

    for i in range(len(df3)):
        if df3['Size'][i]!=a:
            df3['Size'][i]=c
    df3['Size']=df3['Size'].astype('int')
    df=df3
    df.head(n=72)
    df2=df.nsmallest(10,'C')
    df2=df2.loc[df2['C']<0]
    df2['Size2']=b
    df3=pd.merge(df, df2[['Local Authority id','Size2']], on= 'Local Authority id', how='left')
    df=df3
    df.head(n=72)
    for i in range(len(df3)):
        if df3['Size2'][i]==b:
            df3['Size'][i]=b
    df = df3.drop(columns = ['Size2'])
    for i in range(len(df3)):
        if df3['Size2'][i]==b:
            df3['Size'][i]=b
    df = df3.drop(columns = ['Size2'])
    
    import branca.colormap as cm
    rangeFrac = 0.8
    A=abs(df['C'].min())
    B=abs(df['C'].max())
    l=[A,B]
    C=0.15*max(l)
    linear = cm.LinearColormap(['tomato','white','royalblue'], vmin=-C, vmax=C)
    linear2 = cm.LinearColormap(['tomato','royalblue'], vmin=-C, vmax=C)
    linear2 = linear2.to_step(index=[-C, 0, C])
    import folium
    import pandas as pd
    #https://python-visualization.github.io/folium/modules.html

    m = folium.Map(location=[51.9142707824707,-1.108874797821045],zoom_start = 7) 
    # I can add marker one by one on the map
    for i in range(0,len(df)):
        if df.iloc[i]['Size']!=a:
            color= linear(df.iloc[i]['C'])
        else:
            color= linear2(df.iloc[i]['C']),
        folium.Circle(
          location=[df.iloc[i]['Latitude'], df.iloc[i]['Longitude']],
          #popup=data.iloc[i]['name'],
          popup=folium.Popup("<b>"+df.iloc[i]['Local Authority id'].astype('int').astype('str')+': '+ df.iloc[i]['Local Authority name'] + 
                             "</b> <br> C: "+ round(df.iloc[i]['C'],4).astype('str'),
                             max_width=2650),
          radius= df.iloc[i]['Size'].astype('float'),
          fillColor= linear(df.iloc[i]['C']),
          #fillOpacity=,
          #opacity 
          color=color,
          fill=True,
       ).add_to(m)
    # Save it as html
    k = BussTyp['Business Type Description'][1]
    if Bj== 4.067063416869742:
        k+= ' comparing with Global rating average'
    else:
        k+= ' comparing with Business Type rating average'
    title_html = '''
             <h3 align="center" style="font-size:20px"><b>'''+ k +'''</b></h3>
             '''
    m.get_root().html.add_child(folium.Element(title_html))
    m.save(outfile= "C:/Users/Victor.SalasAranda/Desktop/FSA/Sprint-AI-01-FHRS/main/Maps/HTML/Comparing with Global rating average/"+ k + ".html")

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


#### Comparing with Business Type rating average

In [294]:
for j in data['Business Type Description'].unique().tolist():
    BussTyp=data.loc[data['Business Type Description']== j]
    a2=[list(range(0,len(BussTyp)))]
    BussTyp=BussTyp.set_index(a2)
    Bj=BussTyp['Rating Value'].mean()
    dfD = BussTyp[['Local Authority id','Local Authority name']]
    dfD = dfD.drop_duplicates()
    a=10000 #size of the 10 biggest  and 10 smallest values
    b=a 
    c=1500 #size of the normal values
    df = BussTyp.groupby('Local Authority id', as_index=False)['Rating Value'].mean()
    df=pd.merge(df,dfD[['Local Authority id','Local Authority name']], on= 'Local Authority id', how='right')
    df['Latitude'] = BussTyp.groupby('Local Authority id', as_index=False)['Latitude'].mean()['Latitude'].tolist()
    df['Longitude'] = BussTyp.groupby('Local Authority id', as_index=False)['Longitude'].mean()['Longitude'].tolist()
    df['C'] = df['Rating Value'] - Bj
    df2=df.nlargest(10,'C')
    df2['Size']=a
    df3=pd.merge(df, df2[['Local Authority id','Size']], on= 'Local Authority id', how='left')

    for i in range(len(df3)):
        if df3['Size'][i]!=a:
            df3['Size'][i]=c
    df3['Size']=df3['Size'].astype('int')
    df=df3
    df.head(n=72)
    df2=df.nsmallest(10,'C')
    df2['Size2']=b
    df3=pd.merge(df, df2[['Local Authority id','Size2']], on= 'Local Authority id', how='left')
    df=df3
    df.head(n=72)
    for i in range(len(df3)):
        if df3['Size2'][i]==b:
            df3['Size'][i]=b
    df = df3.drop(columns = ['Size2'])
    
    import branca.colormap as cm
    rangeFrac = 0.8
    A=abs(df['C'].min())
    B=abs(df['C'].max())
    l=[A,B]
    C=0.15*max(l)
    linear = cm.LinearColormap(['tomato','white','royalblue'], vmin=-C, vmax=C)
    linear2 = cm.LinearColormap(['tomato','royalblue'], vmin=-C, vmax=C)
    linear2 = linear2.to_step(index=[-C, 0, C])
    import folium
    import pandas as pd
    #https://python-visualization.github.io/folium/modules.html

    m = folium.Map(location=[51.9142707824707,-1.108874797821045],zoom_start = 7) 
    # I can add marker one by one on the map
    for i in range(0,len(df)):
        if df.iloc[i]['Size']!=a:
            color= linear(df.iloc[i]['C'])
        else:
            color= linear2(df.iloc[i]['C']),
        folium.Circle(
          location=[df.iloc[i]['Latitude'], df.iloc[i]['Longitude']],
          #popup=data.iloc[i]['name'],
          popup=folium.Popup("<b>"+df.iloc[i]['Local Authority id'].astype('int').astype('str')+': '+ df.iloc[i]['Local Authority name'] + 
                             "</b> <br> C: "+ round(df.iloc[i]['C'],4).astype('str'),
                             max_width=2650),
          radius= df.iloc[i]['Size'].astype('float'),
          fillColor= linear(df.iloc[i]['C']),
          #fillOpacity=,
          #opacity 
          color=color,
          fill=True,
       ).add_to(m)
    # Save it as html
    k = BussTyp['Business Type Description'][1]
    if Bj== 4.067063416869742:
        k+= ' comparing with Global rating average'
    else:
        k+= ' comparing with Business Type rating average'
    title_html = '''
             <h3 align="center" style="font-size:20px"><b>'''+ k +'''</b></h3>
             '''
    m.get_root().html.add_child(folium.Element(title_html))
    m.save(outfile= "C:/Users/Victor.SalasAranda/Desktop/FSA/Sprint-AI-01-FHRS/main/Maps/HTML/Comparing with Business Type rating average/"+ k + ".html")

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
